[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/corazzon/seoul-bike-analysis/blob/master/crawling.ipynb)

# 크롤링으로 위경도 가져오기
* 따릉이 대여소 정보 가져오기
* 크롤링 위치 : https://www.bikeseoul.com/app/station/moveStationSearchView.do?currentPageNo=1
* colab 경로 : https://colab.research.google.com/github/corazzon/seoul-bike-analysis/blob/master/crawling.ipynb

In [ ]:
# 라이브러리 로드
# requests는 작은 웹브라우저로 웹사이트 내용을 가져온다.
import requests
# BeautifulSoup 을 통해 읽어 온 웹페이지를 파싱한다.
from bs4 import BeautifulSoup as bs
# random은 랜덤한 시간차를 두고 가져오기 위해 사용한다.
import random
# time 으로 시간을 구한다.
import time
# 크롤링 후 결과를 데이터프레임 형태로 보기 위해 불러온다.
import pandas as pd

In [ ]:
# 크롤링 할 사이트
base_url = 'https://www.bikeseoul.com/app/station/moveStationSearchView.do?currentPageNo='

# 전체 대여소 정보를 담아줄 비어있는 리스트를 만든다.
all_stations = []

def crawling_rent_station(pnum):
    response = requests.get( base_url + str(pnum))
        
    if response.status_code != 200:
        return False
    
    soup = bs(response.text, 'html.parser')
    tbody = soup.select('table.psboard1 > tbody')
    
    if not tbody:
        return False
    
    # response의 tbody 값을 전달해 각 대여소의 상세 정보를 받아온다.
    stations = station_info(tbody)
    # www.bikeseoul.com 사이트의 서버에 부담을 덜 주기 위해 시간차를 두고 가져온다.
    time.sleep(random.uniform(1,3))
    if stations:
        # 크롤링 해서 가져온 대여소 정보를 리스트에 담아 줍니다.
        all_stations.extend(stations)

        # 이전 결과가 있다면 페이지번호를 하나씩 더해서 재귀호출을 한다.
        pnum += 1
        # 몇 페이지를 크롤링 하고 있는지 찍어본다. 
        # 다 찍기엔 너무 많아서 10페이지에 한 번씩 출력하도록 한다.
        # 페이지 수를 크롤링 하기 전에 미리 알고 크롤링을 한다면 tqdm을 사용하는 것이 좋다.
        # 2019년 8월 자전거 대여소 페이지수는 307개 이다.
        if pnum % 10 == 0 :
            print(pnum)

        # 같은 함수를 재귀호출해서 실행한다.
        return crawling_rent_station(pnum)
    else:
        return all_stations

In [ ]:
def station_info(tbody):
    if tbody: 
        trs = tbody[0].find_all('tr')
        rent_stations = []
        for tr in trs:
            # 대여소 정보를 담아준다.
            info = []
            # 대여소명
            name = tr.select('td.pl10')[0].get_text(strip=True)
            if '.' in name:
                info.append(name.split('.')[0].strip())
                info.append(name.split('.')[-1].lstrip())
            else:
                # 대여소 정보가 없다면 결측치로 처리한다.
                info.append(pd.np.nan)
                info.append(name)
            # 운영여부
            info.append(tr.select('td.pl10')[1].get_text(strip=True))
            # 주소
            info.append(tr.select('td.mhid')[0].get_text(strip=True))
            # 위도, 경도
            geo = tr.find('a')['param-data'].split(',')
            info.append(geo[0])
            info.append(geo[1])
            rent_stations.append(info)
        return rent_stations
    else :
        return False

In [ ]:
# 전체를 돌려보기 전에 일부만 돌려본다.
crawling_rent_station(306)

In [ ]:
all_stations

In [ ]:
pd.DataFrame(all_stations).sample()

In [ ]:
# 전체 코드를 돌리기 전에 다시 리스트를 비워준다.
all_stations = []
# 아래의 pnum에 1을 입력하면 1페이지부터 끝까지 가져온다.
# pnum에 306 을 306페이지부터 끝까지 가져온다.
# pnum = 1
pnum = 300
# 크롤링 함수를 호출한다.
crawling_rent_station(pnum)

In [ ]:
# 크롤링한 전체 결과를 프린트 한다.
all_stations[:2]

In [ ]:
header = ['대여소번호', '대여소', '상태', '주소', '위도', '경도']
df = pd.DataFrame.from_records(all_stations, columns = header)
df.shape

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
df.to_csv('bike_rent_station.csv', index=False)

In [ ]:
# 파일이 제대로 생성되었는지 확인
pd.read_csv('bike_rent_station.csv').head()